In [41]:
#(thlu = theta leg upper, thll = theta leg lower)


%reset
# DERIVE THE EOMs SYMBOLICALLY ----------------------------------------------------------------------------------------------

# import libraries
import sympy as sym
import numpy as np

sym.init_printing()
from IPython.display import display #for pretty printing

# create symbolic variables

# system parameters
g = sym.symbols('g')
mb,ml1,ml2 = sym.symbols(['m_{body}','m_{leg1}','m_{leg2}']) # mass
lb,ll1,ll2 = sym.symbols(['l_{body}','l_{leg1}','l_{leg2}']) # length
Inb,Inl1,Inl2 = sym.symbols(['I_{body}','I_{leg1}','I_{leg2}']) # moment of intertia

# generalized coordinates
x,y,thb,thlu,thll = sym.symbols(['x','y','\\theta_{body}','\\theta_{leg_upper}','\\theta_{leg_lower}']) 
dx,dy,dthb,dthlu,dthll = sym.symbols(['\dot{x}','\dot{y}','\dot{\\theta}_{body}','\dot{\\theta}_{leg_upper}','\dot{\\theta}_{leg_lower}']) 
ddx,ddy,ddthb,ddthlu,ddthll = sym.symbols(['\ddot{x}','\ddot{y}','\ddot{\\theta}_{body}','\ddot{\\theta}_{leg_upper}','\ddot{\\theta}_{leg_lower}']) 

q = sym.Matrix([[x],[y],[thb],[thlu],[thll]])
dq = sym.Matrix([[dx],[dy],[dthb],[dthlu],[dthll]])
ddq = sym.Matrix([[ddx],[ddy],[ddthb],[ddthlu],[ddthll]])

# forces
# total joint action = actuator + rebound, but that will be dealt with elsewhere
tau_hip,tau_knee,GRFx,GRFy = sym.symbols(['\\tau_hip','\\tau_knee','G_x','G_y']) 

# STEP 1: position vectors ri = [x,y,theta] (world frame)
rb = sym.Matrix([[x],
                [y],
                [thb]])

rl1 = sym.Matrix([[x - 0.5*ll1*sym.sin(thb + thlu)],
                [y - 0.5*ll1*sym.cos(thb + thlu)],
                [thb + thlu]])

rl2 = sym.Matrix([[x - (0.5*ll1)*sym.sin(thb + thlu)-(0.5*ll2)*sym.sin(thb + thlu+thll)],
                [y - (0.5*ll1)*sym.cos(thb + thlu)-(0.5*ll2)*sym.cos(thb + thlu+thll)],
                [thb + thlu+thll]])



# the Jacobians
Jb = rb.jacobian(q)
Jl1 = rl1.jacobian(q)
Jl2 = rl2.jacobian(q)

# STEP 2: generate expressions for the system space velocities from the jacobians
vb = Jb*dq
vl1 = Jl1*dq
vl2 = Jl2*dq

# STEP 3: generate expressions for the kinetic and potential energy
# mass vectors
Mb = sym.Matrix([[mb,mb,Inb]])
Ml1 = sym.Matrix([[ml1,ml1,Inl1]])
Ml2 = sym.Matrix([[ml2,ml2,Inl2]])

T = 0.5*Mb*sym.matrix_multiply_elementwise(vb,vb) + 0.5*Ml1*sym.matrix_multiply_elementwise(vl1,vl1) + 0.5*Ml2*sym.matrix_multiply_elementwise(vl2,vl2)
T = T[0]
V = mb*g*rb[1] + ml1*g*rl1[1] + ml2*g*rl2[1]


# STEP 4: calculate each term of the Lagrange equation
# term 1
Lg1 = sym.zeros(len(q),1)
for i in range(len(q)):
    dT_ddq = sym.Matrix([sym.diff(T,dq[i])]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = sym.Matrix([T]).jacobian(q).transpose() # partial of T in q

# term 4
Lg4 = sym.Matrix([V]).jacobian(q).transpose() # partial of U in q

# STEP 5: generalized forces

# force vectors for each link
tau_b = sym.Matrix([[0],[0],[-tau_hip]])
tau_l1 = sym.Matrix([[0],[0],[tau_hip - tau_knee]])
tau_l2 = sym.Matrix([[0],[0],[tau_knee]])

# F_l1 = sym.Matrix([[-F*sym.sin(thb+thlu)],[F*sym.cos(thb+thlu)],[0]])
# F_l2 = sym.Matrix([[F*sym.sin(thb+thlu)],[-F*sym.cos(thb+thlu)],[0]])

GRF_l2 = sym.Matrix([[GRFx],[GRFy],[0.5*ll2*GRFx*sym.cos(thb+thlu)+0.5*ll2*GRFy*sym.sin(thb+thlu)]])

Q = sym.zeros(len(q),1)
for j in range(len(q)):
    Q[j] = tau_b.transpose()*Jb[:,j]+(tau_l1).transpose()*Jl1[:,j]+(GRF_l2).transpose()*Jl2[:,j]

# AND combine!
EOM = Lg1 - Lg3 + Lg4 - Q

EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    EOMs[j] = EOM[j].simplify()
    
# the simplification step is a little time-consuming so try to avoid re-running this cell if possible.

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [42]:
# Lambdify
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

func_map = {'sin':sin, 'cos':cos} 

sym_list = [g,mb,ml1,ml2,lb,ll1,ll2,Inb,Inl1,Inl2,
            x,y,thb,thlu,thll,
            dx,dy,dthb,dthlu,dthll,
            ddx,ddy,ddthb,ddthlu,ddthll,
            tau_hip,tau_knee,GRFx,GRFy]
            
lambEOM_x = sym.lambdify(sym_list,EOMs[0],modules = [func_map])
lambEOM_y = sym.lambdify(sym_list,EOMs[1],modules = [func_map])
lambEOM_thb = sym.lambdify(sym_list,EOMs[2],modules = [func_map])
lambEOM_thlu = sym.lambdify(sym_list,EOMs[3],modules = [func_map])
lambEOM_thll = sym.lambdify(sym_list,EOMs[4],modules = [func_map])

In [43]:
# rerun from here if you don't want to calculate the EOM's again
if 'm' in globals():
    del m # deletes the model
    
m = ConcreteModel()

# SETS-----------------------------------------------------------------------------------------------------------------------

N = 100
m.N = RangeSet(N) 

links = [('body',1),('leg',1),('leg',2)]
m.L = Set(dimen=2, initialize = links)

DOFs = ['x','y','theta_b','theta_lu','theta_ll'] # generalized coordinates
m.DOF = Set(initialize = DOFs) 

# PARAMETERS-----------------------------------------------------------------------------------------------------------------

m.g = Param(initialize = 9.81)

def get_m(n, lb, ln):
    if lb == 'body':
        return 5.0
    else: return 2.5
m.m = Param(m.L, initialize = get_m) # mass of links

def get_len(n, lb, ln):
    if lb == 'body':
        return 1.0
    else: return 0.5
m.len = Param(m.L, initialize = get_len) # length of links

def calculate_In(m, lb, ln): 
    l = (lb,ln)
    # yes, that does mean you have to rebuild the tuple inside the function. Yes, it is dumb.
    return m.m[l]*m.len[l]**2/12 
m.In = Param(m.L, initialize = calculate_In) # moment of inertia

mbody = sum(m.m[l] for l in links)
BW = mbody*m.g.value

# VARIABLES -----------------------------------------------------------------------------------------------------------------

# system coordinates
m.q = Var(m.N, m.DOF) # position
m.dq = Var(m.N, m.DOF) # velocity
m.ddq = Var(m.N, m.DOF) # acceleration

# bound variables
for n in range(1,N+1):
    for l in links:
        m.q[n,'y'].setlb(0.0)

In [44]:
# TIME AND INTEGRATION

# variable timestep
hm = 0.02 # master timestep
m.h = Var(m.N, bounds = (0.8,1.2))

# Integration constraints 
def BwEuler_p(m,n,dof): # for positions
    if n > 1:
        return m.q[n,dof] == m.q[n-1,dof] + hm*m.h[n]*m.dq[n,dof]
    else:
        return Constraint.Skip #use this to leave out members of a set that the constraint doesn't apply to
m.integrate_p = Constraint(m.N, m.DOF, rule = BwEuler_p)

def BwEuler_v(m,n,dof): # for velocities
    if n > 1:
        return m.dq[n,dof] == m.dq[n-1,dof] + hm*m.h[n]*m.ddq[n-1,dof]
    else:
        return Constraint.Skip 
m.integrate_v = Constraint(m.N, m.DOF, rule = BwEuler_v)

In [45]:
# GROUND INTERACTIONS -------------------------------------------------------------------------------------------------------

# paramters
m.mu = Param(initialize = 1) # friction coefficient

# sign set for positive and negative components
signs = ['ps','ng'] 
m.sgn = Set(initialize = signs)

WDOFs = ['X','Y',"THETA"] # absolute coordinates (see what I mean about switching between frames the whole time...?)
m.WDOF = Set(initialize = WDOFs) 

# variables
m.footp = Var(m.N, m.WDOF, bounds = (0.0,None)) # foot position
m.footv = Var(m.N, m.WDOF, m.sgn, bounds = (0.0,None)) # foot velocity

m.friction_cone = Var(m.N, bounds = (0.0,None))

m.GRF = Var(m.N, m.WDOF, m.sgn, bounds = (0.0,None)) # ground reaction forces

ground_constraints = ['contact','friction','slip_ps','slip_ng'] 
m.ground_constraints = Set(initialize = ground_constraints) # set for indexing ground-related penalties
m.ground_penalty = Var(m.N, m.ground_constraints, bounds = (0.0,None))

# constraints: aux variables
def def_footp(m,n,dof):
    if dof == 'Y':
        L = 0.5*m.len[('leg',1)]  + 0.5*m.len[('leg',2)] # total leg length
        thA = m.q[n,'theta_b'] + m.q[n,'theta_lu'] + m.q[n,'theta_ll'] # absolute leg angle
        return m.footp[n,dof] == m.q[n,'y'] - L*cos(thA)
    else:
        return Constraint.Skip
m.def_footp = Constraint(m.N, m.WDOF, rule = def_footp)

# lambdify the foot velocity
footx = sym.Matrix([x + (0.5*ll1+0.5*ll2)*sym.sin(thb + thlu + thll)])
footdx = footx.jacobian(q)*dq
footdx = footdx[0].simplify()
lamb_footdx = sym.lambdify(sym_list,footdx,modules = [func_map])

def def_footv(m,n,dof):
    if dof == 'X':
        var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_lu'],m.q[n,'theta_ll'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_lu'],m.dq[n,'theta_ll'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_lu'],m.ddq[n,'theta_ll'],
            0,0,0,0]
        return m.footv[n,dof,'ps']-m.footv[n,dof,'ng'] == lamb_footdx(*var_list)
    else:
        return Constraint.Skip
m.def_footv = Constraint(m.N, m.WDOF, rule = def_footv)

def def_friction_cone(m,n):
    return m.friction_cone[n] == m.mu*m.GRF[n,'Y','ps'] - (m.GRF[n,'X','ps'] + m.GRF[n,'X','ng'])
m.def_friction_cone = Constraint(m.N, rule = def_friction_cone)

# constraints: complementarity

# contact
def ground_contact(m,n):
    if n < N:
        return m.ground_penalty[n,'contact'] == m.footp[n+1,'Y']*m.GRF[n,'Y','ps'] 
    else:
        return Constraint.Skip
m.ground_contact = Constraint(m.N, rule = ground_contact)

# friction
def ground_friction(m,n):
    return m.ground_penalty[n,'friction'] == (m.footv[n,'X','ps']+m.footv[n,'X','ng'])*m.friction_cone[n]
m.ground_friction = Constraint(m.N, rule = ground_friction)

# slipping
def ground_slip_ps(m,n):
    return m.ground_penalty[n,'slip_ps'] == m.footv[n,'X','ps']*m.GRF[n,'X','ps']
m.ground_slip_ps = Constraint(m.N, rule = ground_slip_ps)

def ground_slip_ng(m,n):
    return m.ground_penalty[n,'slip_ng'] == m.footv[n,'X','ng']*m.GRF[n,'X','ng']
m.ground_slip_ng = Constraint(m.N, rule = ground_slip_ng)

# bound contact forces at last node
for dof in WDOFs:
    for sgn in signs:
        m.GRF[N,dof,sgn].value = 0
        m.GRF[N,dof,sgn].fixed = True

In [32]:
# HARD JOINT STOPS ----------------------------------------------------------------------------------------------------------

# sets
joints = ['hip','knee']
m.J = Set(initialize = joints)

joint_constraints = ['up','lo'] # set of joint penalties
m.joint_constraints = Set(initialize = joint_constraints)

# parameters
hip_bound = [-np.pi/2,np.pi/2]
m.hip_bound = Param(m.joint_constraints, initialize = {'up':hip_bound[1],'lo':hip_bound[0]}) 

knee_bound = [0.0,0.5]
m.knee_bound = Param(m.joint_constraints, initialize = {'up':knee_bound[1],'lo':knee_bound[0]})

# we can bound the joint coordinates directly
for n in range(1,N+1):
    m.q[n,'theta_lu'].setlb(hip_bound[0])
    m.q[n,'theta_lu'].setub(hip_bound[1])
    m.q[n,'theta_ll'].setlb(hip_bound[0])
    m.q[n,'theta_ll'].setub(hip_bound[1])

# variables
m.tau_a = Var(m.N, bounds = (-2,2)) # actuator torque at hip
#m.tau_r1 = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound torque

m.tau_b = Var(m.N, bounds = (-2,2)) # actuator torque at knee
#m.tau_r2 = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound torque


m.F_a = Var(m.N, bounds = (-3,3)) # actuator force at knee
#m.F_r = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound force (acts parallel to the leg)

m.joint_penalty = Var(m.N, m.J, m.joint_constraints, bounds = (0.0,None))

# # complementarity
# def hip_limits(m,n,jc):
#     if n < N:
#         if jc == 'up':
#             # NEXT angle
#             return m.joint_penalty[n,'hip',jc] == (m.hip_bound['up'] - m.q[n+1,'theta_lu']-m.q[n+1,'theta_ll'])*m.tau_r1[n,'ng']
#         else:
#             return m.joint_penalty[n,'hip',jc] == (m.q[n+1,'theta_lu'] + m.q[n+1,'theta_ll']- m.hip_bound['lo'])*m.tau_r1[n,'ps']
#     else:
#         return Constraint.Skip
# m.hip_limits = Constraint(m.N, m.joint_constraints, rule = hip_limits)

# def knee_limits(m,n,jc):
#     if n < N:
#         if jc == 'up':
#             # NEXT distance
#             return m.joint_penalty[n,'knee',jc] == (m.knee_bound['up'] )*m.F_r[n,'ng']
#         else:
#             return m.joint_penalty[n,'knee',jc] == ( - m.knee_bound['lo'])*m.F_r[n,'ps']
#     else:
#         return Constraint.Skip
# m.knee_limits = Constraint(m.N, m.joint_constraints, rule = knee_limits)

# #bound contact forces at last node
# for sgn in signs:
#     m.F_r[N,sgn].value = 0
#     m.F_r[N,sgn].fixed = True

# for sgn in signs:
#     m.tau_r1[N,sgn].value = 0
#     m.tau_r1[N,sgn].fixed = True
#     m.tau_r2[N,sgn].value = 0
#     m.tau_r2[N,sgn].fixed = True

In [35]:
# HIGH DROP -----------------------------------------------------------------------------------------------------------------

# initial condition
m.q[1,'x'].value = 0.0
m.q[1,'y'].value = 3.0
m.q[1,'theta_b'].value = 0.0
m.q[1,'theta_lu'].value = 0.0
m.q[1,'theta_ll'].value = 0.0

for dof in DOFs:
    m.q[1,dof].fixed = True
    m.dq[1,dof].value = 0.0
    m.dq[1,dof].fixed = True
    
for n in range(1,N+1):
    m.F_a[n].value = 0.0
    m.F_a[n].fixed = True
    m.tau_a[n].value = 0.0
    m.tau_a[n].fixed = True
    
m.pprint()

24 Set Declarations
    DOF : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'x', 'y', 'theta_b', 'theta_lu', 'theta_ll'}
    F_r_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :  N*sgn :  200 : {(1, 'ps'), (1, 'ng'), (2, 'ps'), (2, 'ng'), (3, 'ps'), (3, 'ng'), (4, 'ps'), (4, 'ng'), (5, 'ps'), (5, 'ng'), (6, 'ps'), (6, 'ng'), (7, 'ps'), (7, 'ng'), (8, 'ps'), (8, 'ng'), (9, 'ps'), (9, 'ng'), (10, 'ps'), (10, 'ng'), (11, 'ps'), (11, 'ng'), (12, 'ps'), (12, 'ng'), (13, 'ps'), (13, 'ng'), (14, 'ps'), (14, 'ng'), (15, 'ps'), (15, 'ng'), (16, 'ps'), (16, 'ng'), (17, 'ps'), (17, 'ng'), (18, 'ps'), (18, 'ng'), (19, 'ps'), (19, 'ng'), (20, 'ps'), (20, 'ng'), (21, 'ps'), (21, 'ng'), (22, 'ps'), (22, 'ng'), (23, 'ps'), (23, 'ng'), (24, 'ps'), (24, 'ng'), (25, 'ps'), (25, 'ng'), (26, 'ps'), (26, 'ng'), (27, 'ps'), (27, 'ng'), (28, 'ps'), (28, 'ng'), (29, 'ps')

         (46, 'THETA') :   0.0 :  None :  None : False :  True :  Reals
             (46, 'X') :   0.0 :  None :  None : False :  True :  Reals
             (46, 'Y') :   0.0 :  None :  None : False :  True :  Reals
         (47, 'THETA') :   0.0 :  None :  None : False :  True :  Reals
             (47, 'X') :   0.0 :  None :  None : False :  True :  Reals
             (47, 'Y') :   0.0 :  None :  None : False :  True :  Reals
         (48, 'THETA') :   0.0 :  None :  None : False :  True :  Reals
             (48, 'X') :   0.0 :  None :  None : False :  True :  Reals
             (48, 'Y') :   0.0 :  None :  None : False :  True :  Reals
         (49, 'THETA') :   0.0 :  None :  None : False :  True :  Reals
             (49, 'X') :   0.0 :  None :  None : False :  True :  Reals
             (49, 'Y') :   0.0 :  None :  None : False :  True :  Reals
         (50, 'THETA') :   0.0 :  None :  None : False :  True :  Reals
             (50, 'X') :   0.0 :  None :  None : False :  True :

         (92, 'theta_ll') : -1.5707963267948966 :  None : 1.5707963267948966 : False :  True :  Reals
         (92, 'theta_lu') : -1.5707963267948966 :  None : 1.5707963267948966 : False :  True :  Reals
                (92, 'x') :                None :  None :               None : False :  True :  Reals
                (92, 'y') :                 0.0 :  None :               None : False :  True :  Reals
          (93, 'theta_b') :                None :  None :               None : False :  True :  Reals
         (93, 'theta_ll') : -1.5707963267948966 :  None : 1.5707963267948966 : False :  True :  Reals
         (93, 'theta_lu') : -1.5707963267948966 :  None : 1.5707963267948966 : False :  True :  Reals
                (93, 'x') :                None :  None :               None : False :  True :  Reals
                (93, 'y') :                 0.0 :  None :               None : False :  True :  Reals
          (94, 'theta_b') :                None :  None :               None : Fal

        Key        : Lower : Body                                                                                    : Upper : Active
          (1, 'Y') :   0.0 :           footp[1,Y] - (q[1,y] - 0.5*cos(q[1,theta_b] + q[1,theta_lu] + q[1,theta_ll])) :   0.0 :   True
          (2, 'Y') :   0.0 :           footp[2,Y] - (q[2,y] - 0.5*cos(q[2,theta_b] + q[2,theta_lu] + q[2,theta_ll])) :   0.0 :   True
          (3, 'Y') :   0.0 :           footp[3,Y] - (q[3,y] - 0.5*cos(q[3,theta_b] + q[3,theta_lu] + q[3,theta_ll])) :   0.0 :   True
          (4, 'Y') :   0.0 :           footp[4,Y] - (q[4,y] - 0.5*cos(q[4,theta_b] + q[4,theta_lu] + q[4,theta_ll])) :   0.0 :   True
          (5, 'Y') :   0.0 :           footp[5,Y] - (q[5,y] - 0.5*cos(q[5,theta_b] + q[5,theta_lu] + q[5,theta_ll])) :   0.0 :   True
          (6, 'Y') :   0.0 :           footp[6,Y] - (q[6,y] - 0.5*cos(q[6,theta_b] + q[6,theta_lu] + q[6,theta_ll])) :   0.0 :   True
          (7, 'Y') :   0.0 :           footp[7,Y] - (q[7,y] - 

        Key : Lower : Body                                                                   : Upper : Active
          1 :   0.0 :         friction_cone[1] - (GRF[1,Y,ps] - (GRF[1,X,ps] + GRF[1,X,ng])) :   0.0 :   True
          2 :   0.0 :         friction_cone[2] - (GRF[2,Y,ps] - (GRF[2,X,ps] + GRF[2,X,ng])) :   0.0 :   True
          3 :   0.0 :         friction_cone[3] - (GRF[3,Y,ps] - (GRF[3,X,ps] + GRF[3,X,ng])) :   0.0 :   True
          4 :   0.0 :         friction_cone[4] - (GRF[4,Y,ps] - (GRF[4,X,ps] + GRF[4,X,ng])) :   0.0 :   True
          5 :   0.0 :         friction_cone[5] - (GRF[5,Y,ps] - (GRF[5,X,ps] + GRF[5,X,ng])) :   0.0 :   True
          6 :   0.0 :         friction_cone[6] - (GRF[6,Y,ps] - (GRF[6,X,ps] + GRF[6,X,ng])) :   0.0 :   True
          7 :   0.0 :         friction_cone[7] - (GRF[7,Y,ps] - (GRF[7,X,ps] + GRF[7,X,ng])) :   0.0 :   True
          8 :   0.0 :         friction_cone[8] - (GRF[8,Y,ps] - (GRF[8,X,ps] + GRF[8,X,ng])) :   0.0 :   True
          

        Key : Lower : Body                                                        : Upper : Active
          1 :   0.0 :       ground_penalty[1,slip_ps] - footv[1,X,ps]*GRF[1,X,ps] :   0.0 :   True
          2 :   0.0 :       ground_penalty[2,slip_ps] - footv[2,X,ps]*GRF[2,X,ps] :   0.0 :   True
          3 :   0.0 :       ground_penalty[3,slip_ps] - footv[3,X,ps]*GRF[3,X,ps] :   0.0 :   True
          4 :   0.0 :       ground_penalty[4,slip_ps] - footv[4,X,ps]*GRF[4,X,ps] :   0.0 :   True
          5 :   0.0 :       ground_penalty[5,slip_ps] - footv[5,X,ps]*GRF[5,X,ps] :   0.0 :   True
          6 :   0.0 :       ground_penalty[6,slip_ps] - footv[6,X,ps]*GRF[6,X,ps] :   0.0 :   True
          7 :   0.0 :       ground_penalty[7,slip_ps] - footv[7,X,ps]*GRF[7,X,ps] :   0.0 :   True
          8 :   0.0 :       ground_penalty[8,slip_ps] - footv[8,X,ps]*GRF[8,X,ps] :   0.0 :   True
          9 :   0.0 :       ground_penalty[9,slip_ps] - footv[9,X,ps]*GRF[9,X,ps] :   0.0 :   True
         1

        Key               : Lower : Body                                                              : Upper : Active
           (2, 'theta_b') :   0.0 :           q[2,theta_b] - (q[1,theta_b] + 0.02*h[2]*dq[2,theta_b]) :   0.0 :   True
          (2, 'theta_ll') :   0.0 :        q[2,theta_ll] - (q[1,theta_ll] + 0.02*h[2]*dq[2,theta_ll]) :   0.0 :   True
          (2, 'theta_lu') :   0.0 :        q[2,theta_lu] - (q[1,theta_lu] + 0.02*h[2]*dq[2,theta_lu]) :   0.0 :   True
                 (2, 'x') :   0.0 :                             q[2,x] - (q[1,x] + 0.02*h[2]*dq[2,x]) :   0.0 :   True
                 (2, 'y') :   0.0 :                             q[2,y] - (q[1,y] + 0.02*h[2]*dq[2,y]) :   0.0 :   True
           (3, 'theta_b') :   0.0 :           q[3,theta_b] - (q[2,theta_b] + 0.02*h[3]*dq[3,theta_b]) :   0.0 :   True
          (3, 'theta_ll') :   0.0 :        q[3,theta_ll] - (q[2,theta_ll] + 0.02*h[3]*dq[3,theta_ll]) :   0.0 :   True
          (3, 'theta_lu') :   0.0 :        q[3,t

        Key               : Lower : Body                                                                : Upper : Active
           (2, 'theta_b') :   0.0 :          dq[2,theta_b] - (dq[1,theta_b] + 0.02*h[2]*ddq[1,theta_b]) :   0.0 :   True
          (2, 'theta_ll') :   0.0 :       dq[2,theta_ll] - (dq[1,theta_ll] + 0.02*h[2]*ddq[1,theta_ll]) :   0.0 :   True
          (2, 'theta_lu') :   0.0 :       dq[2,theta_lu] - (dq[1,theta_lu] + 0.02*h[2]*ddq[1,theta_lu]) :   0.0 :   True
                 (2, 'x') :   0.0 :                            dq[2,x] - (dq[1,x] + 0.02*h[2]*ddq[1,x]) :   0.0 :   True
                 (2, 'y') :   0.0 :                            dq[2,y] - (dq[1,y] + 0.02*h[2]*ddq[1,y]) :   0.0 :   True
           (3, 'theta_b') :   0.0 :          dq[3,theta_b] - (dq[2,theta_b] + 0.02*h[3]*ddq[2,theta_b]) :   0.0 :   True
          (3, 'theta_ll') :   0.0 :       dq[3,theta_ll] - (dq[2,theta_ll] + 0.02*h[3]*ddq[2,theta_ll]) :   0.0 :   True
          (3, 'theta_lu') :   0.